In [1]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
RESULTS_DIR = "results"
ASSETS = ["Wind1", "Wind2", "Solar"]
PRODUCTS = ['RT_HUB','RT_NODE','DA_HUB','DA_NODE']

plt.rcParams["figure.figsize"] = (9, 4.8)
df_port = pd.read_csv(f"{RESULTS_DIR}/portfolio_fixed_prices_p75.csv")
df_pgrid = pd.read_csv(f"{RESULTS_DIR}/portfolio_fixed_prices_pgrid.csv")
df_break = pd.read_csv(f"{RESULTS_DIR}/portfolio_breakdown_p75.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'results/portfolio_fixed_prices_p75.csv'

In [ ]:

# %%
# Load portfolio-level tables

with open(f"{RESULTS_DIR}/run_meta.json","r") as f:
    meta = json.load(f)

display(df_port.head(6))
display(df_break.head(6))

# %%
# 1) Fixed prices (P75) by product and asset
pivot = df_port.pivot_table(index="product", columns="asset", values="fixed_price")
ax = pivot.reindex(PRODUCTS).plot(kind="bar")
ax.set_title(f"Fixed Prices at P{int(meta['p_level']*100)}")
ax.set_ylabel("$/MWh")
ax.legend(title="Asset")
plt.tight_layout(); plt.show()

# %%
# 2) P-grid sensitivity curves per product (all assets)
for prod in PRODUCTS:
    sub = df_pgrid[df_pgrid["product"] == prod].copy()
    if sub.empty: 
        continue
    plt.figure()
    for a in ASSETS:
        s2 = sub[sub["asset"] == a]
        if s2.empty:
            continue
        s2 = s2.sort_values("p_level")
        plt.plot(s2["p_level"], s2["fixed_price"], marker="o", label=a)
    plt.title(f"{prod} — Fixed Price vs Risk Appetite (p-level)")
    plt.xlabel("p-level"); plt.ylabel("$/MWh")
    plt.grid(True, alpha=0.3); plt.legend()
    plt.tight_layout(); plt.show()

# %%
# 3) Price breakdown (waterfall-like stacked bars): A/B/C/D/E if available, else expected vs risk premium
# We anticipated ('A_hub_capture','B_basis','C_da_rt','D_neg_rule','E_risk'), but fall back gracefully.
cols_AB = ['A_hub_capture','B_basis','C_da_rt','D_neg_rule','E_risk']
available_AB = [c for c in cols_AB if c in df_break.columns]

for prod in PRODUCTS:
    sub = df_break[df_break["product"] == prod].copy()
    if sub.empty:
        continue

    plt.figure()
    if set(available_AB) == set(cols_AB):
        # Plot the five-step breakdown
        bottom = np.zeros(len(sub))
        for c, lbl in zip(cols_AB, ['Hub','Basis','DA-RT','Neg','Risk']):
            vals = sub[c].astype(float).values
            plt.bar(sub["asset"], vals, bottom=bottom, label=lbl)
            bottom = bottom + vals
        plt.title(f"{prod} — Price Breakdown (Gen-weighted + Risk)")
        plt.ylabel("$/MWh"); plt.legend()
    else:
        # Minimal: Expected ref vs risk premium
        if "expected_ref_price" in sub.columns and "risk_premium" in sub.columns:
            width = 0.35
            x = np.arange(len(sub))
            plt.bar(x - width/2, sub["expected_ref_price"].astype(float).values, width, label="Expected Ref")
            plt.bar(x + width/2, sub["risk_premium"].astype(float).values, width, label="Risk Premium")
            plt.xticks(x, sub["asset"])
            plt.title(f"{prod} — Expected vs Risk Premium"); plt.ylabel("$/MWh"); plt.legend()
        else:
            plt.text(0.5, 0.5, "Breakdown columns not found", ha="center")
    plt.tight_layout(); plt.show()

# %%
# 4) Overlay small samples: hub forward (first 3 days) and generation sample (first 3 days) per asset
for asset in ASSETS:
    try:
        hh = pd.read_csv(f"{RESULTS_DIR}/{asset}_hub_fwd_sample_3d.csv")
        hh["ts"] = pd.to_datetime(hh["ts"])
        plt.figure()
        plt.plot(hh["ts"], hh["hub_forward_hourly"], label="Hub Forward (hourly)")
        plt.title(f"{asset} — Hourly Hub Forward (First 3 Days)")
        plt.ylabel("$/MWh"); plt.xticks(rotation=45); plt.legend()
        plt.tight_layout(); plt.show()
    except FileNotFoundError:
        pass

    try:
        gen = pd.read_csv(f"{RESULTS_DIR}/{asset}_gen_sample_3d.csv")
        gen["ts"] = pd.to_datetime(gen["ts"])
        plt.figure()
        plt.step(gen["ts"], gen["gen_mwh"], where="post", label="Gen (scenario sample)")
        plt.title(f"{asset} — Generation (First 3 Days, one scenario)")
        plt.ylabel("MWh"); plt.xticks(rotation=45); plt.legend()
        plt.tight_layout(); plt.show()
    except FileNotFoundError:
        pass

# %%
# 5) Table summary: best product per asset (lowest P* at P-level)
best = (df_port.loc[df_port.groupby("asset")["fixed_price"].idxmin()]
        .sort_values("fixed_price").reset_index(drop=True))
display(best)
